# EVALUATION METRIC

I've chosen 5 evaluation metrics. And the reason for chosing them is because the target or dependent variable is a continuous variable, hence regression model has to be applied on the dataset.
The metrics i've chosen are as follows:-
1. Mean Squared Error- because it's the standard metric for regression model.
2. Root Mean Squared Error- because it gives the error in the same unit and nt a squared one.
3. Root Mean Squared Log Error- because this type of error is even more precise and relevant than the above two.
4. R-suqare and Adjusted R-square to compare the efficiency of the model in various scenarios.

But for the benchmark model, i've simple use root mean square error only.

# BUILDING THE BENCHMARK MODEL

In [86]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [87]:
#reading the dataset
data=pd.read_csv("E:\\internshala data science course\\COURSE 5-ML\\Resources-NYC Taxi Trip Project\\nyc_taxi_trip_duration.csv")

In [88]:
#exploring data

In [89]:
data.shape

(729322, 11)

In [90]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [91]:
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [92]:
data["pickup_time"]=pd.to_datetime(data["pickup_datetime"],format="%Y-%m-%d %H:%M:%S")
data["dropoff_time"]=pd.to_datetime(data["dropoff_datetime"],format="%Y-%m-%d %H:%M:%S")

In [93]:
data["pickup_hour"]=data["pickup_time"].dt.hour
data["pickup_weekday"]=data["pickup_time"].dt.weekday

In [94]:
data["pickup_hour"]

0         16
1         23
2         17
3          9
4          6
          ..
729317    13
729318     0
729319    18
729320     9
729321    17
Name: pickup_hour, Length: 729322, dtype: int64

In [95]:
def time_of_pickup(x):
    if x in range(4,7):
        return "early morning"
    elif x in range(7,10):
        return "morning"
    elif x in range(10,16):
        return "peak hours"
    elif x in range(16,19):
        return "evening"
    elif x in range(19,22):
        return "night"
    else:
        return "late night"
def weekday_of_pickup(x):
    if x<5:
        return "weekday"
    else:
        return "weekend"

In [96]:
data["pickup_hour"]=data["pickup_hour"].apply(time_of_pickup)
data["pickup_weekday"]=data["pickup_weekday"].apply(weekday_of_pickup)

In [97]:
data["pickup_hour"].value_counts()

peak hours       211031
late night       145715
night            129472
evening          115864
morning           95340
early morning     31900
Name: pickup_hour, dtype: int64

In [98]:
data["pickup_weekday"].value_counts()

weekday    521388
weekend    207934
Name: pickup_weekday, dtype: int64

### Shuffling and creating test and train data

In [99]:
from sklearn.utils import shuffle 
data=shuffle(data,random_state=42)

In [100]:
div=int(data.shape[0]/4)
#dividing data into train and test in 3:1 ratio
train=data.loc[:3*div+1,:]
test=data.loc[3*div+1:]

In [101]:
train.shape

(589451, 15)

In [102]:
test.shape

(139872, 15)

In [103]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_time,dropoff_time,pickup_hour,pickup_weekday
469114,id2380741,2,2016-05-21 10:40:14,2016-05-21 10:51:11,1,-73.981796,40.762035,-73.972267,40.781265,N,657,2016-05-21 10:40:14,2016-05-21 10:51:11,peak hours,weekend
694852,id3946961,2,2016-01-08 18:49:27,2016-01-08 18:52:42,5,-73.980965,40.747677,-73.982704,40.741161,N,195,2016-01-08 18:49:27,2016-01-08 18:52:42,evening,weekday
696324,id0833913,1,2016-05-22 00:54:10,2016-05-22 01:08:10,1,-73.951065,40.782722,-73.867691,40.833664,N,840,2016-05-22 00:54:10,2016-05-22 01:08:10,late night,weekend
356496,id1336849,1,2016-06-11 10:32:12,2016-06-11 10:38:50,1,-73.987625,40.762791,-73.973518,40.762909,N,398,2016-06-11 10:32:12,2016-06-11 10:38:50,peak hours,weekend
645318,id1610858,1,2016-04-03 10:45:51,2016-04-03 10:57:13,3,-73.964333,40.792503,-73.988609,40.758369,N,682,2016-04-03 10:45:51,2016-04-03 10:57:13,peak hours,weekend


In [104]:
test.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_time,dropoff_time,pickup_hour,pickup_weekday
546991,id2240736,1,2016-05-25 07:59:16,2016-05-25 08:05:02,1,-73.991364,40.732590,-74.000526,40.742283,N,346,2016-05-25 07:59:16,2016-05-25 08:05:02,morning,weekday
43126,id1423404,1,2016-01-18 12:17:13,2016-01-18 12:21:13,2,-73.966225,40.768059,-73.967606,40.763073,N,240,2016-01-18 12:17:13,2016-01-18 12:21:13,peak hours,weekday
641450,id1317268,2,2016-03-02 18:39:01,2016-03-02 18:50:12,1,-73.994926,40.766018,-74.004219,40.742523,N,671,2016-03-02 18:39:01,2016-03-02 18:50:12,evening,weekday
611380,id3335546,1,2016-04-06 19:17:20,2016-04-06 19:18:03,1,-73.974388,40.793781,-73.976006,40.792339,N,43,2016-04-06 19:17:20,2016-04-06 19:18:03,night,weekday
62690,id2174190,2,2016-06-21 18:35:31,2016-06-21 18:40:56,3,-73.963440,40.798557,-73.979736,40.777878,N,325,2016-06-21 18:35:31,2016-06-21 18:40:56,evening,weekday


### Simple mean of trip duration

In [105]:
test["mean_trip_duration"]=train["trip_duration"].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\626072653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["mean_trip_duration"]=train["trip_duration"].mean()


In [106]:
#calculating root-mean-squared error

In [107]:
from sklearn.metrics import mean_squared_error as MSE
rms_error=MSE(test["trip_duration"],test["mean_trip_duration"],squared=False)
rms_error

3205.464962628823

### RMSE depending on passenger count

In [108]:
pass_count=pd.pivot_table(train,values="trip_duration",index=["passenger_count"],aggfunc=np.mean)
pass_count

,trip_duration
passenger_count,
0,334.733333
1,919.236280
2,1000.159939
3,1034.393923
4,1028.731304
5,1078.708147
6,1080.694100


In [109]:
test["pass_count_mean"]=0
for i in train["passenger_count"].unique():
    test["pass_count_mean"][test["passenger_count"]==i]=train["trip_duration"][train["passenger_count"]==i].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3882188638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pass_count_mean"]=0
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3882188638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pass_count_mean"][test["passenger_count"]==i]=train["trip_duration"][train["passenger_count"]==i].mean()
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3882188638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [110]:
pass_count_error=MSE(test["trip_duration"],test["pass_count_mean"],squared=False)
pass_count_error

3205.1370715718454

### RMSE depending on vendor ID

In [111]:
ven_ID=pd.pivot_table(train,values="trip_duration",index=["vendor_id"],aggfunc=np.mean)
ven_ID

,trip_duration
vendor_id,
1,838.174354
2,1049.946134


In [112]:
test["ven_ID_mean"]=0
for i in train["vendor_id"].unique():
    test["ven_ID_mean"][test["vendor_id"]==i]=train["trip_duration"][train["vendor_id"]==i].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\2017410572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["ven_ID_mean"]=0
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\2017410572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["ven_ID_mean"][test["vendor_id"]==i]=train["trip_duration"][train["vendor_id"]==i].mean()
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\2017410572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [113]:
ven_ID_error=MSE(test["trip_duration"],test["ven_ID_mean"],squared=False)
ven_ID_error

3203.4437862808136

### RMSE depending on store and fwd flag

In [114]:
flag_type=pd.pivot_table(train,values="trip_duration",index=["store_and_fwd_flag"],aggfunc=np.mean)
flag_type

,trip_duration
store_and_fwd_flag,
N,950.792228
Y,1094.609831


In [115]:
test["flag_type_mean"]=0
for i in train["store_and_fwd_flag"].unique():
    test["flag_type_mean"][test["store_and_fwd_flag"]==i]=train["trip_duration"][train["store_and_fwd_flag"]==i].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\1145867833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["flag_type_mean"]=0
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\1145867833.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["flag_type_mean"][test["store_and_fwd_flag"]==i]=train["trip_duration"][train["store_and_fwd_flag"]==i].mean()
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\1145867833.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [116]:
flag_type_error=MSE(test["trip_duration"],test["flag_type_mean"],squared=False)
flag_type_error

3205.4474618943213

### Simple mean depending on pickup time and pickup weekday

### pickuptime:

In [117]:
pickup_time_type=pd.pivot_table(train,values="trip_duration",index=["pickup_hour"],aggfunc=np.mean)
pickup_time_type

,trip_duration
pickup_hour,
early morning,791.338436
evening,1032.112641
late night,926.383999
morning,898.623545
night,878.037473
peak hours,1018.396096


In [118]:
test["pickup_time_type_mean"]=0
for i in train["pickup_hour"].unique():
    test["pickup_time_type_mean"][test["pickup_hour"]==i]=train["trip_duration"][train["pickup_hour"]==i].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3961660041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pickup_time_type_mean"]=0
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3961660041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pickup_time_type_mean"][test["pickup_hour"]==i]=train["trip_duration"][train["pickup_hour"]==i].mean()
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\3961660041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [119]:
pickup_hour_type_error=MSE(test["trip_duration"],test["pickup_time_type_mean"],squared=False)
pickup_hour_type_error

3205.0919690020723

### pickup weekday

In [120]:
pickup_weekday_type=pd.pivot_table(train,values="trip_duration",index=["pickup_weekday"],aggfunc=np.mean)
pickup_weekday_type

,trip_duration
pickup_weekday,
weekday,970.547577
weekend,904.115785


In [121]:
test["pickup_weekday_type_mean"]=0
for i in train["pickup_weekday"].unique():
    test["pickup_weekday_type_mean"][test["pickup_weekday"]==i]=train["trip_duration"][train["pickup_weekday"]==i].mean()

C:\Users\acer\AppData\Local\Temp\ipykernel_2744\4293583534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pickup_weekday_type_mean"]=0
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\4293583534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pickup_weekday_type_mean"][test["pickup_weekday"]==i]=train["trip_duration"][train["pickup_weekday"]==i].mean()
C:\Users\acer\AppData\Local\Temp\ipykernel_2744\4293583534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [122]:
pickup_weekday_type_error=MSE(test["trip_duration"],test["pickup_weekday_type_mean"],squared=False)
pickup_weekday_type_error

3205.328997223053